In [1]:
import sys
import os
import cv2
import numpy as np
from tqdm import tqdm 
import json
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.append(os.path.dirname(os.getcwd()))
from pkg.teeth_detector import detect_teeth

In [2]:
DATA_DIR = "../experiments/data/teeth_detector"      
RESULTS_DIR = "../experiments/results/teeth_detector"
UPSAMPLE_SCALE = 2.5
DRAW_CONFIDENCE = False
CONFIDENCE_THRESHOLD = 0.8  

In [3]:
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)

In [4]:
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
image_files = [
    f for f in os.listdir(DATA_DIR) 
    if any(f.lower().endswith(ext) for ext in image_extensions)
]

In [5]:
print(f"Найдено изображений для обработки: {len(image_files)}")
if not image_files:
    raise ValueError(f"В папке {DATA_DIR} не найдено изображений")

Найдено изображений для обработки: 24


In [6]:
all_confidences = []

for img_file in tqdm(image_files, desc="Обработка изображений"):
    img_path = os.path.join(DATA_DIR, img_file)
    base_name = Path(img_file).stem
    
    image = cv2.imread(img_path)
    if image is None:
        print(f"Предупреждение: не удалось загрузить {img_file}")
        continue
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    orig_h, orig_w = image_rgb.shape[:2]
    
    new_w, new_h = int(orig_w * UPSAMPLE_SCALE), int(orig_h * UPSAMPLE_SCALE)
    vis_image = cv2.resize(image_rgb, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    scale_x, scale_y = new_w / orig_w, new_h / orig_h
    
    bboxes = detect_teeth(image_rgb)
    confidences = [bbox[4] for bbox in bboxes]
    all_confidences.extend(confidences)
    
    filtered_bboxes = [bbox for bbox in bboxes if bbox[4] >= CONFIDENCE_THRESHOLD]
    
    scaled_bboxes = []
    for bbox in filtered_bboxes:
        x1, y1, x2, y2, conf = bbox
        x1_s = int(x1 * scale_x)
        y1_s = int(y1 * scale_y)
        x2_s = int(x2 * scale_x)
        y2_s = int(y2 * scale_y)
        scaled_bboxes.append((x1_s, y1_s, x2_s, y2_s, conf))
    
    print(f"\n{img_file}: обнаружено {len(bboxes)} зубов")
    print(f"Значения уверенности: {[round(conf, 4) for conf in confidences]}")
    print(f"Средняя уверенность: {np.mean(confidences):.4f}")
    
    stats = {
        "original_image": img_file,
        "total_teeth_detected": len(bboxes),
        "teeth_after_threshold": len(filtered_bboxes),
        "confidence_threshold": CONFIDENCE_THRESHOLD,
        "confidences": confidences,
        "average_confidence": float(np.mean(confidences)) if confidences else 0.0
    }
    
    json_path = os.path.join(RESULTS_DIR, f"{base_name}.json")
    with open(json_path, 'w') as f:
        json.dump(stats, f, indent=4)
    
    for i, (x1, y1, x2, y2, conf) in enumerate(scaled_bboxes):
        cv2.rectangle(vis_image, (x1, y1), (x2, y2), (0, 255, 0), 3)
        
        if DRAW_CONFIDENCE:
            label = f"{conf:.2f}"
            cv2.putText(vis_image, label, (x1, y1 - 5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 3)
    
    output_path = os.path.join(RESULTS_DIR, f"{base_name}.jpg")
    cv2.imwrite(output_path, cv2.cvtColor(vis_image, cv2.COLOR_RGB2BGR))

Обработка изображений:   4%|██▌                                                         | 1/24 [00:00<00:15,  1.49it/s]


1016.png: обнаружено 8 зубов
Значения уверенности: [0.9126, 0.8981, 0.8893, 0.8876, 0.8816, 0.8717, 0.8687, 0.8603]
Средняя уверенность: 0.8837



Обработка изображений:   8%|█████                                                       | 2/24 [00:01<00:12,  1.77it/s]


1017.png: обнаружено 8 зубов
Значения уверенности: [0.9096, 0.909, 0.9063, 0.9017, 0.8955, 0.8794, 0.8663, 0.8429]
Средняя уверенность: 0.8888



Обработка изображений:  12%|███████▌                                                    | 3/24 [00:01<00:10,  1.94it/s]

...


1037.png: обнаружено 25 зубов
Значения уверенности: [0.9091, 0.909, 0.9074, 0.9045, 0.9039, 0.902, 0.9013, 0.9003, 0.8974, 0.8973, 0.8966, 0.8951, 0.8936, 0.8918, 0.8891, 0.8888, 0.8866, 0.8842, 0.8789, 0.8766, 0.8757, 0.8581, 0.8483, 0.848, 0.8269]
Средняя уверенность: 0.8868



Обработка изображений:  96%|████████████████████████████████████████████████████████▌  | 23/24 [00:11<00:00,  2.05it/s]


1038.png: обнаружено 22 зубов
Значения уверенности: [0.9228, 0.9214, 0.909, 0.9086, 0.9075, 0.9056, 0.9007, 0.8996, 0.8958, 0.8921, 0.8898, 0.889, 0.8847, 0.8845, 0.8779, 0.8778, 0.8777, 0.8735, 0.8666, 0.8592, 0.8578, 0.8533]
Средняя уверенность: 0.8889


Обработка изображений: 100%|███████████████████████████████████████████████████████████| 24/24 [00:12<00:00,  1.95it/s]


1039.png: обнаружено 26 зубов
Значения уверенности: [0.9104, 0.8985, 0.8955, 0.8904, 0.884, 0.8829, 0.8818, 0.8805, 0.8803, 0.8795, 0.8785, 0.8781, 0.8707, 0.8697, 0.8656, 0.8634, 0.8622, 0.8616, 0.8553, 0.8552, 0.8543, 0.8535, 0.8417, 0.8389, 0.8266, 0.8255]
Средняя уверенность: 0.8686


In [ ]:
if all_confidences:
    print("\n" + "="*50)
    print("ОБЩАЯ СТАТИСТИКА ПО ВСЕМ ИЗОБРАЖЕНИЯМ")
    print(f"Всего обнаружено зубов: {len(all_confidences)}")
    print(f"Средняя уверенность: {np.mean(all_confidences):.4f}")
    print(f"Минимальная уверенность: {min(all_confidences):.4f}")
    print(f"Максимальная уверенность: {max(all_confidences):.4f}")
    print("="*50)
else:
    print("\nНе обнаружено ни одного зуба во всех изображениях.")